In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

# Self-Supervised pre-training for ASR

This notebook is a basic tutorial for pre-training a model using the self-supervised approach. With this approach, we use a training objective that does not require our dataset to be labeled, which significantly reduces the difficulty of collecting data for pre-training the model. After pre-training our encoder in this way, we can use it in a CTC or RNNT ASR model.

The approach we will use for pre-training our models is represented in the following diagram:

 ![SSL diagram](https://raw.githubusercontent.com/NVIDIA/NeMo/main/tutorials/asr/images/contrastive_ssl.png)

We first mask parts of our input using SpecAugment.  The model is then trained to solve a contrastive task of distinguishing the latent representation of the masked time steps from several sampled distractors. Since our encoders also contain stride blocks which reduce the length of the inputs, in order to obtain target representations we combine several consecutive time steps. They are then passed through a quantizer, which has been found to help with contrastive pre-training.

# Preparing our data

## Downloading dataset

In order to demonstrate how to pre-train the model, we will use the AN4 dataset. Note: this is dataset is much smaller than one that we would typically want to use for self-supervised training, however it will suffice for this tutorial. This dataset also contains transcriptions, but they will be ignored for self-supervised pre-training.

Before we get started, let's download and prepare the dataset. The utterances are available as `.sph` files, so we will need to convert them to `.wav` for later processing. If you are not using Google Colab, please make sure you have [Sox](http://sox.sourceforge.net/) installed for this step--see the "Downloads" section of the linked Sox homepage. (If you are using Google Colab, Sox should have already been installed in the setup cell at the beginning.)

In [ ]:
# This is where the an4/ directory will be placed.
# Change this if you don't want the data to be extracted in the current directory.
data_dir = '.'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

# Download the dataset. This will take a few moments...
print("******")
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'http://www.speech.cs.cmu.edu/databases/an4/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

if not os.path.exists(data_dir + '/an4/'):
    # Untar and convert .sph to .wav (using sox)
    tar = tarfile.open(an4_path)
    tar.extractall(path=data_dir)

    print("Converting .sph to .wav...")
    sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
    for sph_path in sph_list:
        wav_path = sph_path[:-4] + '.wav'
        cmd = ["sox", sph_path, wav_path]
        subprocess.run(cmd)
print("Finished conversion.\n******")

You should now have a folder called `an4` that contains `etc/an4_train.transcription`, `etc/an4_test.transcription`, audio files in `wav/an4_clstk` and `wav/an4test_clstk`, along with some other files we will not need.

Now we can load and take a look at the data. As an example, file `cen2-mgah-b.wav` is a 2.6 second-long audio recording of a man saying the letters "G L E N N" one-by-one. To confirm this, we can listen to the file:

In [ ]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
example_file = data_dir + '/an4/wav/an4_clstk/mgah/cen2-mgah-b.wav'
audio, sample_rate = librosa.load(example_file)

ipd.Audio(example_file, rate=sample_rate)

## Creating Data Manifests

The first thing we need to do now is to create manifests for our training and evaluation data, which will contain the metadata of our audio files. NeMo data sets take in a standardized manifest format where each line corresponds to one sample of audio, such that the number of lines in a manifest is equal to the number of samples that are represented by that manifest. 


A line must contain the path to an audio file, and the duration of the audio sample. For labeled datasets it will also contain the corresponding transcript (or path to a transcript file). Even though for self-supervised pre-training the transcript is unnecessary, we will still add it to our manifest, since we will also be using this dataset to demonstrate fine-tuning later on.

Here's an example of what one line in a NeMo-compatible manifest might look like:
```
{"audio_filepath": "path/to/audio.wav", "duration": 3.45, "text": "this is a nemo tutorial"}
```

We can build our training and evaluation manifests using `an4/etc/an4_train.transcription` and `an4/etc/an4_test.transcription`, which have lines containing transcripts and their corresponding audio file IDs:
```
...
<s> P I T T S B U R G H </s> (cen5-fash-b)
<s> TWO SIX EIGHT FOUR FOUR ONE EIGHT </s> (cen7-fash-b)
...
```

In [ ]:
# --- Building Manifest Files --- #
import json

# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path):
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(')-1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(
                    data_dir, wav_path,
                    file_id[file_id.find('-')+1 : file_id.rfind('-')],
                    file_id + '.wav')

                duration = librosa.core.get_duration(filename=audio_path)

                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
train_transcripts = data_dir + '/an4/etc/an4_train.transcription'
train_manifest = data_dir + '/an4/train_manifest.json'
if not os.path.isfile(train_manifest):
    build_manifest(train_transcripts, train_manifest, 'an4/wav/an4_clstk')
    print("Training manifest created.")

test_transcripts = data_dir + '/an4/etc/an4_test.transcription'
test_manifest = data_dir + '/an4/test_manifest.json'
if not os.path.isfile(test_manifest):
    build_manifest(test_transcripts, test_manifest, 'an4/wav/an4test_clstk')
    print("Test manifest created.")
print("***Done***")

# Self-supervised pre-training

## Setting up the config

First, let's download the default config for ssl pre-training of Citrinet-1024, and the corresponding supervised training config, which we will use for fine-tuning later.

In [ ]:
## Grab the configs we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/citrinet_ssl/citrinet_ssl_1024.yaml
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/citrinet/citrinet_1024.yaml


Since this config is for a very large model, we will modify it to make a much smaller version for the purpose of this tutorial by reducing the number of channels and the number of sub-blocks in each block, as well as reducing masking and weight decay.

In [ ]:
from omegaconf import OmegaConf
import torch

config_path = './configs/citrinet_ssl_1024.yaml'

cfg = OmegaConf.load(config_path)

cfg.model.model_defaults.filters = 256
cfg.model.model_defaults.repeat = 1
cfg.model.model_defaults.enc_final = 256

cfg.model.spec_augment.freq_masks = 0
cfg.model.spec_augment.time_masks = 5

cfg.model.optim.weight_decay = 0
cfg.model.optim.sched.warmup_steps = 500

cfg.model.train_ds.manifest_filepath = "/content/an4/train_manifest.json"
cfg.model.train_ds.batch_size = 16

cfg.model.validation_ds.manifest_filepath = "/content/an4/test_manifest.json"
cfg.model.validation_ds.batch_size = 16

cfg.trainer.max_epochs = None
cfg.trainer.max_steps = 2000 
#in practice you will usually want a much larger amount of pre-training steps
cfg.trainer.log_every_n_steps = 100

if torch.cuda.is_available():
  cfg.trainer.gpus = 1
  cfg.trainer.accelerator = 'dp'
else:
  cfg.trainer.gpus = 0

cfg.exp_manager.exp_dir = "/content/exp"
cfg.exp_manager.name = "pre_trained"
cfg.exp_manager.use_datetime_version = False
cfg.exp_manager.create_tensorboard_logger = False
cfg.exp_manager.resume_if_exists = True
cfg.exp_manager.resume_ignore_no_checkpoint = True
cfg.exp_manager.checkpoint_callback_params.save_best_model = True

The parameters that are relevant to the self-supervised decoder and loss can be found in cfg.model.decoder and cfg.model loss. The default parameters tend to work well, so we will keep them as is for this tutorial.

In [ ]:
print(cfg.model.decoder)
print(cfg.model.loss)

Now we will can create the config object.

In [ ]:
cfg = OmegaConf.to_container(cfg, resolve=True)
cfg = OmegaConf.create(cfg)

## Pre-training the model

We can now create the model based on our config and start pre-training with pytorch lightning. In NeMo you can use the examples/asr/speech_pre_training.py script for this, which contains the following lines:

In [ ]:
import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.asr.models.ssl_models import SpeechEncDecSelfSupervisedModel
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

from nemo.collections.asr.models.ssl_models import SpeechEncDecSelfSupervisedModel
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

trainer = pl.Trainer(**cfg.trainer)
exp_manager(trainer, cfg.get("exp_manager", None))
asr_model = SpeechEncDecSelfSupervisedModel(cfg=cfg.model, trainer=trainer)

trainer.fit(asr_model)

# Fine-tuning with pre-trained encoder

Now that we have pre-trained our encoder, we can fine-tune our model with a supervised objective. Normally the dataset used for pre-training will be large and unlabeled, while the one used for fine-tuning will be a smaller labeled dataset. For the purpose of this tutorial we will just use the same dataset for fine-tuning.

## Building tokenizer

In [ ]:
!mkdir scripts
!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/tokenizers/process_asr_text_tokenizer.py

!python ./scripts/process_asr_text_tokenizer.py \
  --manifest="{data_dir}/an4/train_manifest.json" \
  --data_root="{data_dir}/tokenizers/an4/" \
  --vocab_size=32 \
  --tokenizer="spe" \
  --no_lower_case \
  --spe_type="unigram" \
  --log

## Creating config for fine-tuning with CTC objective

We will now create a config for a ctc-based model with a matching encoder to the model we pre-trained.

In [ ]:
config_path = './configs/citrinet_1024.yaml'

cfg = OmegaConf.load(config_path)

cfg.model.model_defaults.filters = 256
cfg.model.model_defaults.repeat = 1
cfg.model.model_defaults.enc_final = 256
cfg.model.optim.weight_decay = 0
cfg.model.optim.sched.warmup_steps = 500

cfg.model.spec_augment.freq_masks = 0
cfg.model.spec_augment.time_masks = 5

cfg.model.train_ds.manifest_filepath = "/content/an4/train_manifest.json"
cfg.model.train_ds.batch_size = 16

cfg.model.validation_ds.manifest_filepath = "/content/an4/test_manifest.json"
cfg.model.validation_ds.batch_size = 16

cfg.trainer.max_epochs = None
cfg.trainer.max_steps = 2000 

if torch.cuda.is_available():
  cfg.trainer.gpus = 1
  cfg.trainer.accelerator = 'dp'
else:
  cfg.trainer.gpus = 0

cfg.model.tokenizer.dir = data_dir + "/tokenizers/an4/tokenizer_spe_unigram_v32/"  # note this is a directory, not a path to a vocabulary file
cfg.model.tokenizer.type = "bpe"

cfg.exp_manager.exp_dir = "/content/exp"
cfg.exp_manager.name = "fine_tuned"
cfg.exp_manager.use_datetime_version = False
cfg.exp_manager.create_tensorboard_logger = False
cfg.exp_manager.resume_if_exists = True
cfg.exp_manager.resume_ignore_no_checkpoint = True
cfg.exp_manager.checkpoint_callback_params.save_best_model = True

In order to load our encoder, we add the following to the config. We need to set init_strict to False, since the decoder for this model will be different, in order to have the correct output for ctc.

In [ ]:
cfg.init_from_nemo_model="/content/exp/pre_trained/checkpoints/pre_trained.nemo"
cfg.init_strict = False

In [ ]:
cfg = OmegaConf.to_container(cfg, resolve=True)
cfg = OmegaConf.create(cfg)

## Fine-tuning the model

Initializing our model from config:

In [ ]:
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE

trainer = pl.Trainer(**cfg.trainer)
exp_manager(trainer, cfg.get("exp_manager", None))
asr_model = EncDecCTCModelBPE(cfg=cfg.model, trainer=trainer)

Loading our pre-trained checkpoint:

In [ ]:
asr_model.maybe_init_from_pretrained_checkpoint(cfg)

Now we can run fine-tuning.

In [ ]:
trainer.fit(asr_model)